In [2]:
!pip install umap-learn
!pip install scipy
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install missingno
!pip install joblib
!pip install scipy

  Using cached scikit_learn-1.7.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (17 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 81.1 MB/s eta 0:00:00ta 0:00:01
Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 17.3 MB/s eta 0:00:00
Using cached scikit_learn-1.7.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.5 MB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.

In [6]:
!pip install --upgrade numba umap-learn
!pip install numpy==2.2

  Using cached numpy-2.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


# Imports para Preprocesamiento

In [12]:
import pandas as pd
from pathlib import Path
root = Path().resolve()
out = Path(root/'data/')  # Ajusta la ruta según tu caso

# Cargar los DataFrames desde Parquet
df_complete = pd.read_parquet(out / "processed/playlist_tracks_complete.parquet")
df_imputed  = pd.read_parquet(out / "processed/playlist_tracks_imputed.parquet")

# Ahora puedes trabajar con ellos como cualquier DataFrame de pandas
print(df_complete.head())
print(df_imputed.head())

       pid    name  pos       artist_name  \
0  1000000   Party    0         AronChupa   
1  1000000   Party    1         AronChupa   
2  1000000   Party    4             Lorde   
3  1000016  school    0            Alesso   
4  1000016  school    3  American Authors   

                              track_uri  \
0  spotify:track:66U0ASk1VHZsqIkpMjKX3B   
1  spotify:track:5MhsZlmKJG6X5kTHkdwC4B   
2  spotify:track:3G6hD9B2ZHOsgf4WfNu7X1   
3  spotify:track:6WQLkih8nE0JdUCEyLaGnQ   
4  spotify:track:5j9iuo3tMmQIfnEEQOOjxh   

                              artist_uri            track_name  \
0  spotify:artist:5vCOdeiQt9LyzdI87kt5Sh          Little Swing   
1  spotify:artist:5vCOdeiQt9LyzdI87kt5Sh      I'm an Albatraoz   
2  spotify:artist:163tK9Wjr9P9DmM0AVK7lm                  Team   
3  spotify:artist:4AVFqumd2ogHFlRbKIjp1t  Heroes (we could be)   
4  spotify:artist:0MlOPi3zIDMVrfA9R04Fe3   Best Day Of My Life   

                              album_uri  duration_ms        album_name  .

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import joblib
import warnings

# Opcional para Winsorizing (manejo de outliers)
from scipy.stats.mstats import winsorize

# --- Importación Segura de UMAP ---
# Intentamos importar UMAP, si falla por cualquier razón,
# el resto del código no se detendrá.
try:
    import umap.umap_ as umap
    UMAP_AVAILABLE = True
    print("UMAP importado exitosamente.")
except ImportError as e:
    print(f"Advertencia: No se pudo importar UMAP. Error: {e}")
    print("La sección de reducción de dimensionalidad con UMAP será omitida.")
    UMAP_AVAILABLE = False


warnings.filterwarnings('ignore', category=FutureWarning)

# --- DataFrame de partida ---
# Asegúrate de que df_imputed esté cargado de una celda anterior
# y crea una copia para no modificar el original.
df_processed = df_imputed.copy()

print("\nDataFrame inicial para preprocesamiento:")
print(f"Shape: {df_processed.shape}")
df_processed.head()

# ... (El resto del código de preprocesamiento que te proporcioné iría aquí)
# El paso de UMAP al final del script solo se ejecutará si UMAP_AVAILABLE es True.

Advertencia: No se pudo importar UMAP. Error: Numba needs NumPy 2.2 or less. Got NumPy 2.3.
La sección de reducción de dimensionalidad con UMAP será omitida.

DataFrame inicial para preprocesamiento:
Shape: (281000, 95)


,pid,name,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,...,mood_sad_prob,moods_mirex_value,moods_mirex_prob,timbre_value,timbre_prob,tonal_atonal_value,tonal_atonal_prob,voice_instrumental_value,voice_instrumental_prob,was_imputed
0,1000000,Party,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,...,0.527017,Cluster4,0.253382,dark,0.999070,atonal,0.990423,voice,0.917732,0
1,1000000,Party,1,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,...,0.692220,Cluster4,0.334271,bright,0.597991,atonal,0.913942,instrumental,0.751203,0
2,1000000,Party,2,Lorde,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,...,0.661497,None,0.402549,None,0.796112,None,0.963689,None,0.885800,1
3,1000000,Party,3,Lorde,spotify:track:35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,...,0.661497,None,0.402549,None,0.796112,None,0.963689,None,0.885800,1
4,1000000,Party,4,Lorde,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,...,0.765254,Cluster5,0.619994,dark,0.791275,atonal,0.986704,voice,0.988464,0


## 3.1 Eliminar duplicados de tracks

In [14]:
# Seleccionamos las columnas relevantes de las características de audio y el URI
track_feature_cols = ['track_id'] + df_processed.select_dtypes(include=np.number).columns.tolist()
tracks_df_unique = df_processed[track_feature_cols].drop_duplicates(subset=['track_id']).set_index('track_id')

print(f"DataFrame de tracks únicos (sin duplicados por track_id):")
print(f"Shape original de tracks con features: {df_processed.shape[0]}")
print(f"Shape de tracks únicos: {tracks_df_unique.shape[0]}")
tracks_df_unique.head()

DataFrame de tracks únicos (sin duplicados por track_id):
Shape original de tracks con features: 281000
Shape de tracks únicos: 66243


,pid,pos,duration_ms,bpm,energy,danceability_ll,loudness,danceability_prob,gender_prob,genre_dortmund_prob,...,mood_electronic_prob,mood_happy_prob,mood_party_prob,mood_relaxed_prob,mood_sad_prob,moods_mirex_prob,timbre_prob,tonal_atonal_prob,voice_instrumental_prob,was_imputed
track_id,,,,,,,,,,,,,,,,,,,,,
66U0ASk1VHZsqIkpMjKX3B,1000000,0,163809,125.898476,4.910242,1.067135,0.816509,0.941562,0.838504,0.958349,...,0.905014,0.967353,0.660808,0.927280,0.527017,0.253382,0.999070,0.990423,0.917732,0
5MhsZlmKJG6X5kTHkdwC4B,1000000,1,166848,128.037704,4.296187,0.937844,0.785428,0.507221,0.590108,0.991147,...,0.917834,0.986929,0.999164,0.808827,0.692220,0.334271,0.597991,0.913942,0.751203,0
0GZoB8h0kqXn7XFm4Sj06k,1000000,2,232506,117.939562,4.532946,1.045318,0.840784,0.815972,0.716691,0.936630,...,0.846012,0.966654,0.811448,0.771735,0.661497,0.402549,0.796112,0.963689,0.885800,1
35kahykNu00FPysz3C2euR,1000000,3,216600,117.939562,4.532946,1.045318,0.840784,0.815972,0.716691,0.936630,...,0.846012,0.966654,0.811448,0.771735,0.661497,0.402549,0.796112,0.963689,0.885800,1
3G6hD9B2ZHOsgf4WfNu7X1,1000000,4,193058,99.882507,4.392409,1.130975,0.920415,0.999133,0.721461,0.860394,...,0.715190,0.945679,0.774370,0.579097,0.765254,0.619994,0.791275,0.986704,0.988464,0


Nota: Seguiremos trabajando con df_processed para las tareas que involucran playlists, pero tracks_df_unique es el input correcto para un futuro clustering a nivel de canción.

## 3.2 Tratamiento de NA Categóricos

Como identificamos en el EDA, las columnas de texto (categóricas) aún tienen valores nulos. Los imputaremos con la constante "desconocido".

In [15]:
# Identificar columnas categóricas (tipo 'object')
categorical_cols = df_processed.select_dtypes(include=['object', 'category']).columns.tolist()

# El track_id no debe ser imputado, lo removemos si está en la lista
if 'track_id' in categorical_cols:
    categorical_cols.remove('track_id')

# Crear el imputer
imputer_cat = SimpleImputer(strategy='constant', fill_value='desconocido')

# Aplicar el imputer
df_processed[categorical_cols] = imputer_cat.fit_transform(df_processed[categorical_cols])

print("Porcentaje de nulos después de imputar categóricas:")
print(df_processed[categorical_cols].isnull().sum() / len(df_processed) * 100)
df_processed.head()

Porcentaje de nulos después de imputar categóricas:
name                         5.338078
artist_name                  0.000000
track_uri                    0.000000
artist_uri                   0.000000
track_name                   0.000000
album_uri                    0.000000
album_name                   0.000000
mbid                         0.033096
danceability_value          55.837722
gender_value                55.837722
genre_dortmund_value        55.837722
genre_electronic_value      55.837722
genre_rosamerica_value      55.837722
genre_tzanetakis_value      55.837722
ismir04_rhythm_value        55.837722
mood_acoustic_value         55.837722
mood_aggressive_value       55.837722
mood_electronic_value       55.837722
mood_happy_value            55.837722
mood_party_value            55.837722
mood_relaxed_value          55.837722
mood_sad_value              55.837722
moods_mirex_value           55.837722
timbre_value                55.837722
tonal_atonal_value          55.83772

,pid,name,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,...,mood_sad_prob,moods_mirex_value,moods_mirex_prob,timbre_value,timbre_prob,tonal_atonal_value,tonal_atonal_prob,voice_instrumental_value,voice_instrumental_prob,was_imputed
0,1000000,Party,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,...,0.527017,Cluster4,0.253382,dark,0.999070,atonal,0.990423,voice,0.917732,0
1,1000000,Party,1,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,...,0.692220,Cluster4,0.334271,bright,0.597991,atonal,0.913942,instrumental,0.751203,0
2,1000000,Party,2,Lorde,spotify:track:0GZoB8h0kqXn7XFm4Sj06k,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Yellow Flicker Beat - From The Hunger Games: M...,spotify:album:4UEPxQx0cTcYNsE0n32MHV,232506,Yellow Flicker Beat,...,0.661497,None,0.402549,None,0.796112,None,0.963689,None,0.885800,1
3,1000000,Party,3,Lorde,spotify:track:35kahykNu00FPysz3C2euR,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,White Teeth Teens,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,216600,Pure Heroine,...,0.661497,None,0.402549,None,0.796112,None,0.963689,None,0.885800,1
4,1000000,Party,4,Lorde,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,...,0.765254,Cluster5,0.619994,dark,0.791275,atonal,0.986704,voice,0.988464,0
